## Replace doi references with actual links

In [1]:
from bs4 import BeautifulSoup

In [2]:
import requests

In [3]:
import pandas as pd
from collections import Counter

In [4]:
from urllib.parse import urlparse

In [5]:
from functions import log, isInRef

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
headers = { # this is my header
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
}

In [8]:
# dictionary for getting journal name from url
journals_df = pd.read_csv('domain_to_journal.csv', delimiter='\t')
journal_ref = pd.Series(journals_df.journal.values,index=journals_df.domain).to_dict()
# dict: journal_ref['url'] = journal name

In [9]:
df = pd.read_csv('results.2.csv', delimiter= '\t')

In [10]:
df.head()

,link,title,abstract,references,journal,doi
0,https://journals.plos.org/plosone/article?id=1...,"Awareness, perception and perpetration of cybe...","The modern online society requires everyone, e...",['https://doi.org/10.1111/j.1746-1561.2008.003...,PLOS,https://doi.org/10.1371/journal.pone.0267702
1,https://journals.plos.org/plosone/article?id=1...,Validation pipeline for machine learning algor...,A standardized objective evaluation method is ...,"['https://doi.org/10.1109/TMI.2014.2377694', '...",PLOS,https://doi.org/10.1371/journal.pone.0267213
2,https://journals.plos.org/plosone/article?id=1...,Identifying luminal and basal mammary cell spe...,Mammary gland is present in all mammals and us...,"['https://doi.org/10.1002/wdev.35', 'https://d...",PLOS,https://doi.org/10.1371/journal.pone.0267211
3,https://journals.plos.org/plosone/article?id=1...,Using machine learning for the personalised pr...,Revision endoscopic sinus surgery (ESS) is oft...,"['https://doi.org/10.4193/Rhin20.600', 'https:...",PLOS,https://doi.org/10.1371/journal.pone.0267146
4,https://journals.plos.org/plosone/article?id=1...,Underwater acoustic target recognition method ...,To improve the recognition accuracy of underwa...,"['https://doi.org/10.3390/e20040243', 'https:/...",PLOS,https://doi.org/10.1371/journal.pone.0266425


In [11]:
# # all references in flat list
# all_references = []
# for sublist in df.references:
#     sub = eval(sublist) # convert from string
#     for item in sub:        
#         all_references.append(item)    

In [12]:
# initialize journal list
journals = []

In [13]:
# collect an example link for every journal
samples = {}
def get_sample_url(url, domain):
    if not(domain in samples.keys()):
        samples[domain] = url

In [14]:
def main():
    # will replace references column
    all_references = []
    counter = 0; # for knowing progress
    for sublist in df.references:
        sub_references = []
        sub = eval(sublist) # bc its stored as string, parse to list     
        for link in sub:
            try:
                # get request and extract domain
                request = requests.get(link, headers=headers, verify=False, timeout = 5)
                domain = urlparse(request.url).netloc
                # append to master list of domains
                journals.append(domain)
                # collect example article for each domain
                get_sample_url(request.url, domain)
                # check if domain is in reference file as some journal
                if isInRef(domain):            
                    # append to list of references for this article
                    sub_references.append(link+"|"+domain)

                    # counter for progress purposes
                    counter+=1;
                    if counter%25 == 0:
                        print(counter)
                else:
                    # do same thing actually lol
                    sub_references.append(link+"|"+domain)

                    # counter for progress purposes
                    counter+=1;
                    if counter%25 == 0:
                        print(counter)
                        
            except requests.exceptions.Timeout:
                print("Timeout occurred")
                    
                
        # append list of references for this article        
        all_references.append(sub_references)
        
    df.references = all_references
    
    log(f'Ran analyze_ref.3.ipynb succesfully')


In [15]:
main()

25
50
75
100
125
150
175
200
225
250
275
300
325
350
375


In [16]:
df.head()

,link,title,abstract,references,journal,doi
0,https://journals.plos.org/plosone/article?id=1...,"Awareness, perception and perpetration of cybe...","The modern online society requires everyone, e...",[https://doi.org/10.1111/j.1746-1561.2008.0033...,PLOS,https://doi.org/10.1371/journal.pone.0267702
1,https://journals.plos.org/plosone/article?id=1...,Validation pipeline for machine learning algor...,A standardized objective evaluation method is ...,[https://doi.org/10.1109/TMI.2014.2377694|ieee...,PLOS,https://doi.org/10.1371/journal.pone.0267213
2,https://journals.plos.org/plosone/article?id=1...,Identifying luminal and basal mammary cell spe...,Mammary gland is present in all mammals and us...,[https://doi.org/10.1002/wdev.35|onlinelibrary...,PLOS,https://doi.org/10.1371/journal.pone.0267211
3,https://journals.plos.org/plosone/article?id=1...,Using machine learning for the personalised pr...,Revision endoscopic sinus surgery (ESS) is oft...,[https://doi.org/10.4193/Rhin20.600|www.rhinol...,PLOS,https://doi.org/10.1371/journal.pone.0267146
4,https://journals.plos.org/plosone/article?id=1...,Underwater acoustic target recognition method ...,To improve the recognition accuracy of underwa...,[https://doi.org/10.3390/e20040243|www.mdpi.co...,PLOS,https://doi.org/10.1371/journal.pone.0266425


In [17]:
df.to_csv(r'results.3.csv', sep='\t', index = False)

In [18]:
len(journals)

396

In [19]:
counter = Counter(journals)

In [20]:
counter.most_common()

[('linkinghub.elsevier.com', 107),
 ('onlinelibrary.wiley.com', 47),
 ('link.springer.com', 27),
 ('journals.lww.com', 15),
 ('journals.sagepub.com', 14),
 ('journals.plos.org', 14),
 ('www.nature.com', 13),
 ('www.tandfonline.com', 12),
 ('www.frontiersin.org', 12),
 ('doi.org', 12),
 ('ieeexplore.ieee.org', 9),
 ('academic.oup.com', 9),
 ('journals.physiology.org', 9),
 ('www.mdpi.com', 8),
 ('www.hindawi.com', 7),
 ('www.rhinologyjournal.com', 6),
 ('www.liebertpub.com', 5),
 ('doi.apa.org:443', 5),
 ('jamanetwork.com', 4),
 ('breast-cancer-research.biomedcentral.com', 4),
 ('bmcmedinformdecismak.biomedcentral.com', 3),
 ('www.science.org', 3),
 ('www.cambridge.org', 3),
 ('kns.cnki.net', 3),
 ('bmcpublichealth.biomedcentral.com', 2),
 ('pnas.org', 2),
 ('www.karger.com', 2),
 ('aip.scitation.org', 2),
 ('www.acpjournals.org', 2),
 ('validate.perfdrive.com', 2),
 ('www.scielo.br', 2),
 ('journals.aps.org', 2),
 ('content.iospress.com:443', 2),
 ('www.cnki.net', 2),
 ('publications.a

In [21]:
samples

{'onlinelibrary.wiley.com': 'https://onlinelibrary.wiley.com/doi/10.1111/j.1746-1561.2008.00335.x',
 'link.springer.com': 'https://link.springer.com/article/10.1007/s10964-017-0678-4',
 'www.liebertpub.com': 'https://www.liebertpub.com/doi/10.1089/cyber.2012.0207',
 'journals.sagepub.com': 'https://journals.sagepub.com/doi/10.1177/1010539514548759',
 'linkinghub.elsevier.com': 'https://linkinghub.elsevier.com/retrieve/pii/S0145213415001131',
 'jamanetwork.com': 'https://jamanetwork.com/journals/jamapsychiatry/fullarticle/210833',
 'doi.apa.org:443': 'https://doi.apa.org:443/doiLanding?doi=10.1037/a0035618',
 'www.tandfonline.com': 'https://www.tandfonline.com/doi/full/10.1080/13811118.2017.1304304',
 'publications.aap.org': 'https://publications.aap.org/pediatrics/article/127/4/800/65133/The-Impact-of-Social-Media-on-Children-Adolescents',
 'bmcpsychiatry.biomedcentral.com': 'https://bmcpsychiatry.biomedcentral.com/articles/10.1186/s12888-021-03365-1',
 'bjgp.org': 'https://bjgp.org/co